# Developing AI System

### Chatbot

A **chatbot** is a computer program that simulates human conversation, usually through text or voice. With **OpenAI**, a chatbot uses powerful language models like **GPT-4** to understand questions and generate smart, human-like answers.

In simple terms:

> A chatbot with OpenAI can chat, answer questions, summarize info, or help with tasks — almost like talking to a helpful assistant.

It works by sending messages to the model (input) and receiving a response (output), often using the `chat.completions.create()` function from the OpenAI API.



In [17]:
#This version of chatbot can only be running through the console,
#Check code app.py a version of this chatbot deployed on Streamlit Platform for a better experience
#To run the chatbot --> Terminal : streamlit run app.py

import os 
import openai
from time import sleep
from tqdm import tgrange, tqdm_notebook

#Bloc
MESSAGES = [{"role":"system", "content": "You're a Chatbot, your name is AlexBot you've been created by Alexandre Ohayon, you can answer any customer's questions by summarizing your answer."}]
#Dialog Box ask your question to the ChatBot:

try:
    print("AlexBot is ready. Tape 'exit' to quit the conversation")

    #Start a session:
    while True:
        #Enter your question:
        USER_PROMPT = input("Enter your message here: ") #Use the dialog box
        #Tape exist in lower character to quit the conversation with the Bot.
        if USER_PROMPT.lower() == "exit": 
            print("Thanks for choosing AlexBot. See you soon !") #Message from system
            break #Session closed
            
        #If a conversation exists then add user question to the Bloc in line 7:
        MESSAGES.append({"role":"user", "content": USER_PROMPT}) #Add the question into the list:
        user_question = {"role":"user", "content": USER_PROMPT}
        #print(f"User_question : {USER_PROMPT}")

                
        #Pass MESSAGES through the LLM to get response from the AI Agent:
        client = openai.OpenAI(organization=None,
                                project=None, 
                                timeout=60*5, #5 minutes and break after that
                                max_retries=2,
                                api_key=os.getenv("API_KEY"))
            
        #Response from the Intelligent Agent:
        # Add the appropriate parameters to the decorator
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=MESSAGES,
            max_tokens=300,
            temperature=0,
            top_p=0.9,
            stop=["\n"]
        )

        assistant_dict = {"role":"assistant", "content": response.choices[0].message.content}
        print(f"AlexBot : {assistant_dict["content"]}")
        print("--------------------------------------------------")
        #Add the question into the list:
        MESSAGES.append(assistant_dict)

except Exception as e:
    print("Error :", e)

AlexBot is ready. Tape 'exit' to quit the conversation
AlexBot : Hello! How can I assist you today?
--------------------------------------------------
Thanks for choosing AlexBot. See you soon !


### Get response in Json Format 

In [22]:
import openai
import os

# Initialisation du client OpenAI avec la clé API (via variable d'environnement)
client = openai.OpenAI(organization=None,
                                project=None, 
                                timeout=60*5, #5 minutes and break after that
                                max_retries=2,
                                api_key=os.getenv("API_KEY")) # Remplace si besoin

def get_famous_dishes_by_country():
    # Prompt demandé
    prompt = (
        "Donne-moi une liste des 10 plats les plus connus dans le monde, "
        "triée par pays, au format JSON. Pour chaque plat, indique le nom du plat et le pays d'origine. "
        "Utilise ce format : "
        '{ "plats": [ { "nom": "Sushi", "pays": "Japon" }, ... ] }'
    )

    # Appel à l'API avec la méthode chat.completions.create
    response = client.chat.completions.create(
        model="gpt-4o",  # ou "gpt-4"
        messages=[
            {"role": "system", "content": "Tu es un assistant culinaire qui fournit des réponses en JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=500,
        response_format={"type":"json_object"} # Nécessite GPT-4-turbo ou GPT-4o
    )

    return response.choices[0].message.content

# Utilisation
if __name__ == "__main__":
    json_result = get_famous_dishes_by_country()
    print(json_result)


{
  "plats": [
    { "nom": "Sushi", "pays": "Japon" },
    { "nom": "Pizza", "pays": "Italie" },
    { "nom": "Tacos", "pays": "Mexique" },
    { "nom": "Couscous", "pays": "Maroc" },
    { "nom": "Paella", "pays": "Espagne" },
    { "nom": "Croissant", "pays": "France" },
    { "nom": "Hamburger", "pays": "États-Unis" },
    { "nom": "Poutine", "pays": "Canada" },
    { "nom": "Kimchi", "pays": "Corée du Sud" },
    { "nom": "Curry", "pays": "Inde" }
  ]
}


### Function tools 

Function tools (also called tools or function calling in OpenAI) let you connect a chatbot to external functions in your code.

🔧 In simple terms:
Function tools allow the chatbot to call real functions — like searching a database, getting the weather, or running a calculation — based on the user's question.

📌 Example:
If the user says:

"What's the weather in Paris?"

The model can call your get_weather(city) function, get the result, and reply with it.

In [19]:
import os
import openai
import json

system = "You're an AI agent capable of analyzing job offers"
prompt = "Create a simple job offer for a data scientist position in Microsoft in Montreal and salary equal to 120,000$ "

try:
    # Création de l'offre
    client = openai.OpenAI(api_key=os.getenv("API_KEY"))

    response1 = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ]
    )

    job_offer = response1.choices[0].message.content

    # Extraction des informations
    response2 = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": job_offer}
        ],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "extract_job_info",
                    "description": "Extract key information from a job offer: position, city, salary, and company name.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "job_position": {
                                "type": "string",
                                "description": "The job title (e.g., Data Scientist)."
                            },
                            "city": {
                                "type": "string",
                                "description": "City where the job is located."
                            },
                            "salary": {
                                "type": "string",
                                "description": "Salary for the job."
                            },
                            "company_name": {
                                "type": "string",
                                "description": "Company offering the position."
                            }
                        },
                        "required": ["job_position", "city", "salary", "company_name"]
                    }
                }
            }
        ],
        tool_choice={"type": "function", "function": {"name": "extract_job_info"}}
    )

    print("📄 Job Offer:\n", job_offer)
    print("\n------------------------------")

    # Extraire les arguments du tool_call
    tool_call = response2.choices[0].message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    print("🧠 Extracted Info:")
    print(f"📌 Position      : {args['job_position']}")
    print(f"🏙️  City          : {args['city']}")
    print(f"💼 Company Name : {args['company_name']}")
    print(f"💰 Salary        : {args['salary']}")

except Exception as e:
    print("❌ Error:", e)


📄 Job Offer:
 **Job Offer: Data Scientist**

**Company:** Microsoft  
**Location:** Montreal, Quebec, Canada  
**Position:** Data Scientist  
**Salary:** $120,000 CAD per year  

**About the Role:**  
Microsoft is seeking a talented and motivated Data Scientist to join our dynamic team in Montreal. In this role, you will leverage cutting-edge technologies and advanced analytical techniques to drive data-driven decision-making and contribute to impactful projects across the organization.

**Key Responsibilities:**  
- Develop, implement, and maintain machine learning models and algorithms.
- Analyze large datasets to extract actionable insights and business value.
- Collaborate with cross-functional teams to understand their data needs and provide innovative solutions.
- Present findings and recommendations to stakeholders in a clear and concise manner.
- Continuously stay updated with the latest industry trends and advancements in data science.

**Qualifications:**  
- Bachelor’s or Ma

### Build LLM Apps with LangChain

>What is LangChain and Why Use It?

LangChain is a powerful framework for building applications powered by language models like GPT-4. It helps developers connect LLMs with external data (like PDFs, databases, APIs), manage conversation memory, and build complex workflows using tools and agents.

>Why use LangChain?

Because it simplifies the process of building smart, context-aware applications with LLMs — like chatbots, search engines, or autonomous agents — and makes them more scalable, modular, and production-ready.

In [23]:
# pip install langchain
# pip install langchain_openai

#Import LangChain
from langchain_openai import ChatOpenAI
#Build an llm
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("API_KEY"),
    base_url=None, organization=None, seed=None)

prompt="What is LangChain ?"
response = llm.invoke(prompt)
print("\nLLM output:")
print(f"-----------------------")
print(response.content)


LLM output:
-----------------------
LangChain is an open-source framework designed for building applications that incorporate language models. It provides tools and components that make it easier to develop sophisticated applications that can leverage natural language processing (NLP) capabilities. LangChain is particularly useful for creating chatbots, AI assistants, and other applications that require the integration of language models with external data sources and APIs.

Key features of LangChain include:

1. **Modular Design**: LangChain allows developers to compose different components like models, prompt templates, and chains of operations to build complex applications.

2. **Chain Abstractions**: Developers can create chains of operations that combine multiple actions, such as prompting a language model, retrieving information from a database, or making API calls.

3. **Memory**: It provides options to integrate memory into applications, allowing them to remember past interact

### Prompting Hugging Face models

>Prompting Hugging Face Models with LangChain: What & Why

LangChain supports integration with Hugging Face models, allowing you to use open-source LLMs (like BLOOM, Falcon, or Mistral) in your applications.

>What does it mean to prompt Hugging Face models in LangChain?

It means you can send inputs (prompts) to Hugging Face-hosted models via LangChain and process the outputs as part of a chain, agent, or tool-based system.

>Why use LangChain with Hugging Face models?

Because it gives you the flexibility of open-source models while still benefiting from LangChain’s structure — such as chaining logic, memory management, and tool use — without relying solely on OpenAI or closed APIs.


In [24]:
#pip install huggingface_hub
#pip install langchain_huggingface
#pip install transformers
#pip install torch torchvision

from langchain_huggingface import HuggingFacePipeline 

llm = HuggingFacePipeline.from_model_id(
    model_id= "tiiuae/falcon-rw-1b", #Choose a model llm from huggingface hub
    task="text-generation",
    pipeline_kwargs={"max_new_tokens":100}
)


Device set to use mps:0


KeyboardInterrupt: 

In [25]:
#Define the LLM:
llm = ChatOpenAI(model='gpt-4o-mini',
                 api_key=os.getenv("API_KEY")
                 )
#Predict the words following the text in question:
prompt = "Three reasons for using LangChain for LLM application development"
#Get response:
response = llm.invoke(prompt)
#Print response
print(response.content)

LangChain is a powerful framework designed for developing applications using large language models (LLMs). Here are three reasons to consider using LangChain for LLM application development:

1. **Modularity and Composability**: LangChain offers a modular architecture that allows developers to easily compose different components, such as prompt templates, memory management, and chains of operations. This flexibility enables developers to build complex applications by combining simple building blocks, making it easier to iterate on designs and implement varied functionalities.

2. **Integration with External APIs and Tools**: LangChain can seamlessly integrate with various external APIs, databases, and tools, allowing developers to enhance the capabilities of LLMs beyond text generation. This includes integrating with search engines, knowledge bases, and even other machine learning models, enabling the creation of more interactive and context-aware applications.

3. **Built-In Support f

In [14]:
#Import the class for defining Hugging Face pipelines:
from langchain_openai import ChatOpenAI 
from langchain_huggingface import HuggingFacePipeline

#Define the LLM from the Hugging Face model ID:
llm = HuggingFacePipeline.from_model_id(
    model_id="crumb/nano-mistral",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 50}
)
#Define the prompt:
prompt = "What is the capital of France ?"
print(f"Output:  /n {llm.invoke(prompt)}")

Device set to use mps:0


Output:  /n What is the capital of France ?

The capital of France, France, is a province in the French province of France in the southern part of the French province of France. The land is divided into two regions: the provinces of France and the provinces of France.



In [29]:
#Asking for real-time data such as weather:
from langchain_openai import ChatOpenAI

prompt = "What is the weather in Montreal ?"

llm = ChatOpenAI(model='gpt-4o-mini',
                 api_key=os.getenv("API_KEY"))
print(llm.invoke(prompt).content)


I'm unable to provide real-time weather updates. To check the current weather in Montreal, I recommend using a reliable weather website or app. You can find up-to-date information on temperature, conditions, and forecasts there.


In [31]:
# Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("API_KEY"))
prompt = 'Three reasons for using LangChain for LLM application development.'
response = llm.invoke(prompt)
print(response.content)

# Import the class for defining Hugging Face pipelines
from langchain_huggingface import HuggingFacePipeline

# Define the LLM from the Hugging Face model ID
llm = HuggingFacePipeline.from_model_id(
    model_id="crumb/nano-mistral",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 20}
)

prompt = "Hugging Face is"
response = llm.invoke(prompt)
print(response)

LangChain is a versatile framework designed to simplify the development of applications using Large Language Models (LLMs). Here are three key reasons for using LangChain in LLM application development:

1. **Modularity and Extensibility**: LangChain is built with a modular architecture, allowing developers to easily integrate various components such as models, data connectors, and workflows. This modularity facilitates the customization and extension of applications, enabling developers to tweak individual components or swap them out as needed without rewriting the entire application.

2. **Integrated Pipelines**: LangChain provides built-in support for creating end-to-end pipelines for LLM applications. It allows developers to seamlessly chain together different processing steps, such as data retrieval, pre-processing, and post-processing, while leveraging LLMs. This can streamline the development process by automating tasks and ensuring efficient data flow, making it easier to build

Device set to use mps:0


Hugging Face is an exciting way to show your support for the NHS and the NHS that matters.
The


### Integrating ChatPromptTemplate

    🧩 What is LangChain?
LangChain is like a magic tool that helps you talk to AI (like ChatGPT) and build smart apps. It helps you give instructions to AI in a smart and reusable way.

    📝 What is a PromptTemplate in LangChain?
Let’s say you’re building an app that always asks the AI:

“What is a fun fact about ___?”

But instead of writing the whole question every time, you want a shortcut.

In LangChain, you make a PromptTemplate like this:

- from langchain.prompts import PromptTemplate
- template = PromptTemplate.from_template("What is a fun fact about {topic}?")
The {topic} part is like a blank you can fill in later.

    🎉 Why is it cool?
    
You only write the question once.
You can reuse it again and again by changing the topic.
It makes your app or chatbot faster and smarter.

In [43]:
#Import packages from LangChain:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate

#Create a prompt template specifying the AI Behaviour:
template = "You are an artificial intelligence assistant, answer the question. {question}"
prompt_template = PromptTemplate.from_template(
    template=template
    )
#Create the LLM:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("API_KEY"))
print(llm)

#Create a chain to integrate the prompt template and LLM using LCEL (LangChain Expression Language):
llm_chain = prompt_template | llm 
llm_chain

question = "How does LangChain make LLM application development easier ?"
print(llm_chain.invoke(question).content)


client=<openai.resources.chat.completions.completions.Completions object at 0x3bc23c0d0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x3bc23c250> root_client=<openai.OpenAI object at 0x136aee7b0> root_async_client=<openai.AsyncOpenAI object at 0x136aef230> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')
LangChain simplifies LLM (Large Language Model) application development by providing a framework that streamlines the process of building applications around language models. Here are some key ways it accomplishes this:

1. **Modular Components**: LangChain offers various modular components that developers can easily plug into their applications. These include tools for prompt management, chat formats, and ways to handle inputs and outputs.

2. **Prompt Engineering**: It provides utilities for creating and managing prompts, allowing developers to fine-tune their interactions with LLMs without needing to start fro

### ChatModels
Chat roles : **system**, **human** and **ai**

- system	🎬 The director	Sets the rules and tone (e.g., "You are a helpful assistant")
- human	👦 The person talking	Asks questions or gives instructions
- AI (or assistant)	🤖 The actor / chatbot	Replies based on what the human said and the system rules

In [55]:
#Import packages:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

#Create the LLM:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("API_KEY"))

#Create the prompte template:
prompt_template = ChatPromptTemplate.from_messages(
    [
        #Behaviour's artificial intelligent agent (system):
        ("system","You are a geography expert that returns the colors present in a country flag."),
        #User's question (user):
        ("human","France"),
        #AI's answer (assistant/ai):
        ("ai","Blue,White,Red"),
        #User's question (user):
        ("human","{country}")
    ]
)

llm_chain = prompt_template | llm 


In [64]:
# Import packages
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Create the LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("API_KEY"))

# Create the prompt template Chat using ChatPromptTemplate (Conversation History)
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert that returns the colors present in a country flag."),
    ("human", "France"),
    ("ai", "Blue, White, Red"),
    ("human", "{country}")
])

# Build the chain
llm_chain = prompt_template | llm
print(llm_chain)
print(prompt_template)

#Asnwer:
country = "Canada"
response = llm_chain.invoke({"country": country}).content
print(response)

first=ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a geography expert that returns the colors present in a country flag.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='France'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Blue, White, Red'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}'), additional_kwargs={})]) middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x453f82bd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x4

In [75]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

#Quick Start with LangChain:
llm = ChatOpenAI(api_key=os.getenv("API_KEY"), model="gpt-4o-mini", verbose=True)
prompt = "Please tell me which country belongs to North America ?"
response = llm.invoke(input=prompt, config=None, stop="[\n]")
print(response.content)

North America includes several countries. The most prominent ones are:

1. **United States**
2. **Canada**
3. **Mexico**

Additionally, there are smaller countries in the Caribbean and Central America that are part of North America, such as:

4. **Guatemala**
5. **Belize**
6. **Honduras**
7. **El Salvador**
8. **Nicaragua**
9. **Costa Rica**
10. **Panama**

Moreover, the Caribbean countries like Cuba, Jamaica, the Bahamas, and others are also considered part of North America.


In [100]:
from langchain import PromptTemplate

#Using Pipeline LCEL:
template = "You are a specialist in geography. Answer the following question:\n{question}"
prompt_template = PromptTemplate.from_template(
    template=template
)
llm_model = ChatOpenAI(api_key=os.getenv("API_KEY"), model="gpt-4o-mini", verbose=True)
llm_chain = prompt_template | llm_model
print(llm_chain)
question2 = {"question": "Please tell me which countries belong to North America?"}
print(llm_chain.invoke(question).content)

first=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are a specialist in geography. Answer the following question:\n{question}') middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x4543534d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x454352b50>, root_client=<openai.OpenAI object at 0x136e7dfd0>, root_async_client=<openai.AsyncOpenAI object at 0x136e7db70>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
North America is made up of several countries, primarily divided into three regions: 

1. **Northern America**:
   - United States
   - Canada
   - Mexico

2. **Central America**: (considered part of North America geographically)
   - Belize
   - Costa Rica
   - El Salvador
   - Guatemala
   - Honduras
   - Nicaragua
   - Panama

3. **Caribbean**: (also part of the wider North American region)
   - Antigu

In [102]:
# Import packages
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

#Prompt Template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You have an intelligent agent capable of returning the color of a fruit or vegetable."),
    ("human", "Banana"),
    ("ai", "Color : Yellow"),
    ("human", "{user_input}")
])

#Build the chain
llm_model = ChatOpenAI(api_key=os.getenv("API_KEY"), model="gpt-4o-mini")
llm_chain = prompt_template | llm_model

#Response
ai_response = llm_chain.invoke({"user_input":"Grapes"}).content
print(ai_response)

Color: Purple (or green, depending on the variety)


### Few-shot prompting

Few-shot prompting means giving the AI a few examples of how you want it to respond before asking your real question.

In [116]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Create the examples list of dicts
examples = [
  {
    "question": "How many fingers do we have ?",
    "answer": "10"
  },
  {
    "question": "How many eyes do we have?",
    "answer": "2"
  },
  {
    "question": "How many ears do we have?",
    "answer": "2"
  }
]

# Complete the prompt for formatting answers
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

# Create the few-shot prompt
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

prompt = prompt_template.invoke({"input": "How many heads do we have?"})
print(prompt.text)

# Create an OpenAI chat LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("API_KEY"))

# Create and invoke the chain
llm_chain = prompt_template | llm
print(llm_chain.invoke({"input": "How many heads do we have?"}).content)

Question: How many fingers do we have ?
10

Question: How many eyes do we have?
2

Question: How many ears do we have?
2

Question: How many heads do we have?
1


### Sequential chains

In [127]:
#Let's plan an itinary with LangChain:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

#Sequential prompts:
destination_prompt = PromptTemplate(
    input_variables = ["destination"],
    template = "I'm going on vacation to {destination}, can you suggest some activites out there ?"
    )
activities_prompt = PromptTemplate(
    input_variables = ["activities"],
    template = "Please, create an itinary for the top three activities : {activities} "
)
#LLM Model:
llm_model = ChatOpenAI(api_key=os.getenv("API_KEY"), model="gpt-4o-mini")

####Sequential Chain:

#Build the chain:
get_activities = destination_prompt | llm_model | StrOutputParser()

seq_chain = (
    RunnableMap({"activities": get_activities})
    | activities_prompt
    | llm_model
    | StrOutputParser()
)

# 4️⃣ Run the chain
result = seq_chain.invoke({"destination": "Paris"})
print(result)
             

Certainly! Here’s a suggested itinerary focusing on the top three activities in Paris: the Eiffel Tower, the Louvre Museum, and the Seine River Cruise. This itinerary assumes a two-day visit, giving you ample time to enjoy each attraction and some leisure time in the city.

### **Day 1: Eiffel Tower and Seine River Cruise**

**Morning:**
- **Breakfast:** Start your day with a traditional French breakfast at a nearby café—croissants and café au lait are excellent choices.
- **Eiffel Tower:** Arrive early to avoid crowds. Pre-book your tickets online to access the summit. Spend time enjoying the panoramic views of Paris from the top.

**Afternoon:**
- **Lunch:** Have lunch at one of the cafes near the Eiffel Tower. Consider trying a classic French dish like quiche or ratatouille.
- **Leisure Time:** Take a walk along the Champ de Mars or relax on the banks of the Seine.

**Evening:**
- **Seine River Cruise:** Book a sunset cruise to take advantage of the beautiful evening lighting. Many 

### Introduction to LangChain Agents

LangChain agents are a powerful feature that enable large language models (LLMs) to make decisions and take actions based on dynamic reasoning and tool use. If you're just getting started, here's a beginner-friendly introduction to what LangChain agents are, how they work, and when to use them.

🧠 **What is a LangChain Agent ?**
- An agent in LangChain is a component that:
- Receives a user query
- Thinks step by step about how to respond
- Chooses tools (APIs, functions, databases, etc.) to call
- Executes tools as needed
- Returns the final answer to the user

It acts almost like an AI assistant that can use tools to accomplish goals, similar to how a human might use a calculator, a search engine, or a file system.

## ReAct (Reason + Act) agent 

In [140]:
#Calling a simple AI agent to answer user's prompt:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

#Assisant's behaviour:
template = "You are an intelligent agent capable of answering any questions. {question}"
prompt_template = PromptTemplate.from_template(
    template=template
)
#LLM model:
llm_model = ChatOpenAI(api_key=os.getenv("API_KEY"), 
                       model="gpt-4")
#Chain:
llm_chain = prompt_template | llm_model |StrOutputParser()

#Prompt:

print(llm_chain.invoke(input="what is your behaviour and What is the biggest animal on earth ?"))

As an intelligent agent, my behavior involves processing information, learning continuously, and providing accurate answers to the best of my ability. I strive to assist users in getting the information they need efficiently and reliably.

The biggest animal on earth is the blue whale. It can grow up to 100 feet long and weigh as much as 200 tons.
